Problem Statement: [Image Processing #3](https://docs.google.com/document/d/1X9bgNOv5uKe7nrCib8gNzYmNaDYvd9Vnb2-MXZ-30Kw/edit)



In [5]:
#!pip3 install torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 663.1MB 27kB/s 
     |████████████████████████████████| 8.8MB 22.7MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


In [3]:
#!pip install "pillow<7"

     |████████████████████████████████| 2.1MB 4.3MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.0.0


## Setup


In [27]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Data

In [29]:
train=torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor()]))
test=torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor()]))
trainset=torch.utils.data.DataLoader(train, batch_size=10,shuffle=True, num_workers=1)
testset=torch.utils.data.DataLoader(test, batch_size=10, shuffle=False, num_workers=1)

In [30]:
trainset_shape=trainset.dataset.train_data.shape
testset_shape=testset.dataset.test_data.shape

print(trainset_shape, testset_shape)

torch.Size([60000, 28, 28]) torch.Size([10000, 28, 28])


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


## Model Training

In [31]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

In [32]:
network = Net()
print(network)

Net(
  (fc1): Linear(in_features=784, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
)


In [34]:
loss_criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(network.parameters(), lr=0.001)

In [36]:
for epoch in range(5): 
    for data in trainset:  
        X, y = data  
        network.zero_grad()  
        output = network(X.view(-1,784))  
        loss = loss_criterion(output, y)  
        loss.backward()  
        optimizer.step()  
    print(loss) 

tensor(1.3474, grad_fn=<NllLossBackward>)
tensor(0.0130, grad_fn=<NllLossBackward>)
tensor(0.0035, grad_fn=<NllLossBackward>)
tensor(0.0060, grad_fn=<NllLossBackward>)
tensor(0.0643, grad_fn=<NllLossBackward>)


## Model Evaluation

In [40]:
correct = 0
total = 0

with torch.no_grad():
    for data in testset:
        X, y = data
        output = network(X.view(-1,784))
        
        for idx, i in enumerate(output):
               total+=1
               if torch.argmax(i) == y[idx]:
                  correct += 1

print("Accuracy: ", round(correct/total, 2))

Accuracy:  0.97
